# Erstellung des Korpus

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import re

In [2]:
# Metadaten einlesen

md = pd.read_csv('metadata.csv', sep=';', encoding='utf-8')

In [3]:
files = []

for file in os.listdir('Texte'):
    files.append(file)

In [4]:
# Texte einlesen 

data = []

for file in os.listdir('Texte'):
    with open(os.path.join('Texte', file),'r', encoding='utf-8', errors='ignore') as myfile:
        data.append(myfile.read())

In [5]:
# Metadaten und Texte zusammenfügen (matcht den Dokumentnamen)

corpus = pd.DataFrame({'file': files, 'text': data})

df = pd.merge(md, corpus, how='left', left_on='doc', right_on='file')

In [6]:
# Wörter zählen und hinzfügen

df['words'] = df['text'].str.count(r'\w+')

In [7]:
# Spalten neu anordnen und unnötige entfernen

df = df[['doc', 'source', 'author', 'title', 'year', 'text_type', 'text', 'words']]

In [8]:
df.head()

,doc,source,author,title,year,text_type,text,words
0,diacorisCampanile2.txt,DiaCoris,Achille Campanile,Gli asparagi e l'immortalità dell'anima,1974,espositivo,"nonché interessarsi , non riesce addirittura a...",165.0
1,LISCampanile1.txt,LIS,Achille Campanile,Chi non fa rirecitare non s'aspetti di ritrasm...,1960,stampa,TV e attori s ' accapigliano su una cosa che n...,1179.0
2,LISCampanile2.txt,LIS,Achille Campanile,Belle prodezze/ Perry/ mettere nel sacco lo st...,1960,stampa,"Sempre novità , alla TV , sempre idee fresche ...",1188.0
3,LISCampanile3.txt,LIS,Achille Campanile,Peggio del monopolio il monopolio del monopolio,1960,stampa,"Nella severa aula della Consulta , davanti all...",1579.0
4,LISCampanile4.txt,LIS,Achille Campanile,Il triplice maledetto destino die refusi origi...,1960,stampa,Voi dite che questa grande invenzione che è la...,1214.0


In [9]:
df.shape 

(1636, 8)

In [10]:
# KWIC-Listen 
# (vorher einzeln in einheitliche Form gebracht, siehe Vorbereitungscode in den Ordnern der jeweiligen Korpora)

directory = 'KWIC'
kwic_files = glob.glob(os.path.join(directory, '*.csv'))

In [11]:
# alle KWIC-Listen einlesen 

frames = []

for kwic_file in kwic_files:
    df_new = pd.read_csv(kwic_file, sep=',', encoding='utf-8')
    frames.append(df_new)

In [12]:
# zu einem Dataframe zusammenfügen 

df_big = pd.concat(frames)

In [13]:
df_big.head()

,doc,source,author,title,year,text_type,text,words
0,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.dica33.it/argomenti/tossicologia/dr...,http://www.dica33.it/argomenti/tossicologia/dr...,2005.0,stampa,"Il programma delle attività prevede il 27, 28 ...",24
1,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://qn.quotidiano.net/chan/tuttocalcio%3a53...,http://qn.quotidiano.net/chan/tuttocalcio%3a53...,2005.0,stampa,Anche i nostri lettori non capiscono la sua li...,9
2,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.kataweb.it/sport/sportdetail.jsp%3f...,http://www.kataweb.it/sport/sportdetail.jsp%3f...,2005.0,stampa,Tutto questo mentre di fronte ai nostri occhi ...,28
3,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.kataweb.it/sport/sportdetail.jsp%3f...,http://www.kataweb.it/sport/sportdetail.jsp%3f...,2005.0,stampa,"Su una cosa sono tutti d'accordo, amici e nemi...",19
4,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://it.sports.yahoo.com/050319/8/7179.html,http://it.sports.yahoo.com/050319/8/7179.html,2005.0,stampa,"Ogni partita è stata difficile, e siamo orgogl...",17


In [14]:
df_big.shape

(244597, 8)

In [15]:
# KWIC-Listen und selbst erstelltes Korpus zu einem zusammenfügen

df_bigger = pd.concat([df_big, df], axis=0, join='outer', ignore_index=False)

In [16]:
df_bigger.head()

,doc,source,author,title,year,text_type,text,words
0,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.dica33.it/argomenti/tossicologia/dr...,http://www.dica33.it/argomenti/tossicologia/dr...,2005.0,stampa,"Il programma delle attività prevede il 27, 28 ...",24.0
1,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://qn.quotidiano.net/chan/tuttocalcio%3a53...,http://qn.quotidiano.net/chan/tuttocalcio%3a53...,2005.0,stampa,Anche i nostri lettori non capiscono la sua li...,9.0
2,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.kataweb.it/sport/sportdetail.jsp%3f...,http://www.kataweb.it/sport/sportdetail.jsp%3f...,2005.0,stampa,Tutto questo mentre di fronte ai nostri occhi ...,28.0
3,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.kataweb.it/sport/sportdetail.jsp%3f...,http://www.kataweb.it/sport/sportdetail.jsp%3f...,2005.0,stampa,"Su una cosa sono tutti d'accordo, amici e nemi...",19.0
4,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://it.sports.yahoo.com/050319/8/7179.html,http://it.sports.yahoo.com/050319/8/7179.html,2005.0,stampa,"Ogni partita è stata difficile, e siamo orgogl...",17.0


In [17]:
df_bigger.shape

(246233, 8)

In [18]:
# Text von Tags befreien

def parser(text):
    return re.sub('<[^<]+?>', '', str(text))

df_bigger['text'] = df_bigger.text.apply(parser)

In [19]:
# NaN durch 0 ersetzen (nur für den Fall, um spätere Fehler zu vermeiden)

df_bigger = df_bigger.fillna(0)

In [20]:
# chronologisch ordnen

df_bigger = df_bigger.sort_values(by=['year'])

In [21]:
# Zeitraum zuordnen

df_bigger['period'] = ['1700-1750' if x >= 1700 and x <= 1750 
                    else '1751-1800' if x >= 1751 and x <= 1800
                    else '1801-1825' if x >= 1801 and x <= 1825
                    else '1826-1850' if x >= 1826 and x <= 1850
                    else '1851-1875' if x >= 1851 and x <= 1875
                    else '1876-1900' if x >= 1876 and x <= 1900
                    else '1901-1925' if x >= 1901 and x <= 1925
                    else '1926-1950' if x >= 1926 and x <= 1950
                    else '1951-1975' if x >= 1951 and x <= 1975
                    else '1976-2000' if x >= 1976 and x <= 2000
                    else '2001-2021' if x >= 2001 and x <= 2021
                    else '>1700' 
                    for x in pd.to_numeric(df_bigger['year'])]

In [22]:
# unbrauchbare Zeilen entfernen

df_bigger = df_bigger[df_bigger.period != '>1700']

In [23]:
# Spalten neu anordnen und unnötige entfernen

df_final = df_bigger[['doc', 'source', 'author', 'title', 'year', 'period', 'text_type', 'text', 'words']]

In [24]:
df_final

,doc,source,author,title,year,period,text_type,text,words
1081,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0
626,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0
1341,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0
1565,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0
447,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0
...,...,...,...,...,...,...,...,...,...
2083,JSItremendamente.csv,JSI,rollingstone.it,rollingstone.it,2021.0,2001-2021,web,monopolizzato. Finché c''è varietà di modo di...,42.0
350,JSIdomani_randomsample.csv,JSI,ilsecoloxix.it,ilsecoloxix.it,2021.0,2001-2021,web,durante tutto il periodo estivo interessate da...,31.0
576,JSIieri_randomsample.csv,JSI,sanmarinortv.sm,sanmarinortv.sm,2021.0,2001-2021,web,"Busignani, è la legge sul consumo: ""Darebbe la...",38.0
1289,JSIraramente_randomsample.csv,JSI,repubblica.it,repubblica.it,2021.0,2001-2021,web,"una malattia, muoversi nel terreno incerto del...",38.0


In [25]:
# speichern

df_final.to_csv('Korpus/corpus_complete.csv', encoding='utf-8', index=False)